<a href="https://colab.research.google.com/github/UrvashiiThakur/practiceGit/blob/main/17April.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Q1. What is Gradient Boosting Regression?
Gradient Boosting Regression is an ensemble machine learning technique that combines multiple weak learners, typically decision trees, to form a strong predictive model for regression tasks. The technique builds the model in a stage-wise fashion, where each subsequent model corrects the errors of the previous ones by fitting to the negative gradient of the loss function with respect to the predictions.

### Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy
Here is a basic implementation of a gradient boosting regression algorithm from scratch:

```python
import numpy as np

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []

    def _fit_tree(self, X, y):
        from sklearn.tree import DecisionTreeRegressor
        tree = DecisionTreeRegressor(max_depth=self.max_depth)
        tree.fit(X, y)
        return tree

    def fit(self, X, y):
        self.models = []
        # Initialize with mean prediction
        y_pred = np.full(y.shape, np.mean(y))
        for _ in range(self.n_estimators):
            residuals = y - y_pred
            tree = self._fit_tree(X, residuals)
            y_pred += self.learning_rate * tree.predict(X)
            self.models.append(tree)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for tree in self.models:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

# Simple example
if __name__ == "__main__":
    from sklearn.datasets import make_regression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, r2_score

    # Create a simple regression dataset
    X, y = make_regression(n_samples=100, n_features=1, noise=0.1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Train the model
    gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
    gbr.fit(X_train, y_train)

    # Evaluate the model
    y_pred = gbr.predict(X_test)
    print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
    print(f"R-squared: {r2_score(y_test, y_pred)}")
```

### Q3. Experiment with different hyperparameters
To optimize the performance, we can use Grid Search to find the best hyperparameters.

```python
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

gbr = GradientBoostingRegressor()
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Train with the best parameters
gbr_optimized = GradientBoostingRegressor(**best_params)
gbr_optimized.fit(X_train, y_train)
y_pred_optimized = gbr_optimized.predict(X_test)

print(f"Optimized Mean Squared Error: {mean_squared_error(y_test, y_pred_optimized)}")
print(f"Optimized R-squared: {r2_score(y_test, y_pred_optimized)}")
```

### Q4. What is a weak learner in Gradient Boosting?
A weak learner in Gradient Boosting is typically a simple model that performs slightly better than random guessing. In the context of Gradient Boosting, decision stumps (trees with a single split) or shallow decision trees are often used as weak learners. The idea is to combine many such weak learners to form a strong predictive model.

### Q5. What is the intuition behind the Gradient Boosting algorithm?
The intuition behind Gradient Boosting is to sequentially add models to an ensemble in a way that each new model corrects the errors made by the previous models. This is done by fitting each new model to the negative gradient of the loss function with respect to the current ensemble's predictions, thus gradually reducing the overall error.

### Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?
Gradient Boosting builds an ensemble of weak learners by iteratively training each new model to predict the residual errors of the combined ensemble of previous models. The predictions of each new model are then scaled by a learning rate and added to the existing ensemble predictions.

### Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting algorithm?
1. **Initialize the model** with a constant value, typically the mean of the target variable for regression.
2. **Iteratively add models** to the ensemble:
   - Compute the negative gradient of the loss function with respect to the current predictions (these are the pseudo-residuals).
   - Fit a weak learner (e.g., a decision tree) to the pseudo-residuals.
   - Scale the predictions of the weak learner by a learning rate.
   - Update the ensemble predictions by adding the scaled predictions of the new weak learner.
3. **Combine the models** by summing their scaled predictions to form the final model.

Here's a more detailed step-by-step for a simple Gradient Boosting algorithm:

1. **Initialization:** Start with an initial prediction, typically the mean of the target values.
   \[ F_0(x) = \arg\min_\gamma \sum_{i=1}^{n} L(y_i, \gamma) \]
   For mean squared error, this is simply the mean of \( y \).

2. **For each iteration \( m = 1 \) to \( M \):**
   - Compute the pseudo-residuals (negative gradient of the loss function):
     \[ r_i^{(m)} = -\left[\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\right]_{F(x_i) = F_{m-1}(x_i)} \]
   - Fit a weak learner (e.g., decision tree) to the pseudo-residuals:
     \[ h_m(x) = \arg\min_h \sum_{i=1}^{n} \left(r_i^{(m)} - h(x_i)\right)^2 \]
   - Update the model with the learning rate \( \eta \):
     \[ F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x) \]

3. **Output the final model**:
   \[ F(x) = F_M(x) \]

By iteratively fitting weak learners to the residuals of the previous model, Gradient Boosting effectively reduces the overall error and improves predictive performance.